<div style = "font-family:Georgia;
              font-size:2.5vw;
              color:lightblue;
              font-weight:normal;
              text-align:center;
              background:url('./text_images/Title Background.gif') no-repeat center; background-size:cover)">
    <br>
    <br>
    Principal Component Analysis (PCA)
    <br>
    <br>
    <br>
</div>

# 引言

在之前的课程中，你学习了**主成分分析 (PCA)** 的核心概念，并学习了特征向量和特征值。在此 notebook 中，在向风险因子模型应用 PCA 之前，我们将了解如何使用 PCA **降维**。简而言之，降维是指减少解释数据所用的变量。

首先简单介绍下降维，然后使用 Scikit-Learn 的 PCA 类降低随机相关数据的维度，并可视化数据的主成分。


# 降维

主成分分析的主要应用之一是降低高度相关的数据的维度。例如，假设数据如下所示：
<br>
<figure>
  <img src = "./text_images/1.png" width = 80% style = "border: thin silver solid; padding: 10px">
      <figcaption style = "text-align: center; font-style: italic">Fig 1. - Highly Correlated Data.</figcaption>
</figure> 
<br>

可以看出，这个二维数据由两个变量描述：$X$ 和 $Y$。但是，你会发现所有数据点都紧挨着一条直线：

<br>
<figure>
  <img src = "./text_images/2.png" width = 80% style = "border: thin silver solid; padding: 10px">
      <figcaption style = "text-align: center; font-style: italic">Fig 2. - Direction of Biggest Variation.</figcaption>
</figure> 
<br>

我们发现大部分变化都沿着这条直线发生。意味着我们可以通过仅查看数据沿着这条线的分布情况，解释数据的大部分变化。所以，我们可以通过将数据点投射到这条直线上，将数据从二维降到一维：

<br>
<figure>
  <img src = "./text_images/3.png" width = 80% style = "border: thin silver solid; padding: 10px">
      <figcaption style = "text-align: center; font-style: italic">Fig 3. - Projected Points.</figcaption>
</figure> 
<br>

这样会将描述数据所需的变量数量从 2 个减少到 1 个，因为只需一个数字就能指定数据点在直线上的位置。所以，描述二维图表的 2 个变量将替换成一个表示一维线性关系的新变量。

<br>
<figure>
  <img src = "./text_images/4.png" width = 80% style = "border: thin silver solid; padding: 10px">
      <figcaption style = "text-align: center; font-style: italic">Fig 4. - Data Reduced to 1D.</figcaption>
</figure> 
<br>


需要注意的是，这个新的变量和维度不一定要有任何特定的含义。例如，在原始的二维图表中，$X$ 和 $Y$ 可能表示股票收益率，但是降维后，新的变量和维度不一定就要表示任何此类含义。新的变量和维度只是抽象工具，使我们能够以更紧凑的方式表达数据。虽然在某些情形下，这些新的变量和维度可能代表真实的量，但是未必会这样。

降低相关性数据的维度适合任何维数，即可以将 $N$-维数据将为 $k$-维数据，其中 $k < N$。正如之前提到的，PCA 是主要降维工具之一。为了查看降维原理，我们会向某个随机相关的数据应用 PCA。在下个部分，我们将根据给定的相关性创建一个随机数据集。


# 创建数据集

在此部分，我们将学习如何创建随机相关的数据。在以下代码中，我们将使用 `utils` 模块的 `utils.create_corr_data()` 函数创建随机相关的数据。我们专门针对此 notebook 创建了 `utils` 模块，其中包含一些有用的函数。你可以查看该模块，看看函数的运行原理。在以下代码中，你可以为数据设定数据范围和相关性。然后，这些参数将传递给 `utils.create_corr_data()` 函数来创建数据。最后，我们使用 `utils` 模块的 `utils.plot_data()` 函数绘制数据。你可以研究下随着相关性的变化，数据是如何变化的。注意，相关性是从 -1 到 1 的数字。相关性为 `0` 表示数据点之间没有关系，相关性为 `1` 和 `-1` 分别表示数据点完全正相关和完全负相关。对于 `corr = +/-1`，所有数据点将在一条直线上。

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

import utils

# Set the default figure size
plt.rcParams['figure.figsize'] = [10.0, 6.0]

# Set data range
data_min = 10
data_max = 80

# Set the amount of correlation. The correlation is anumber in the closed interval [0,1].
corr = 0.8

# Create correlated data
X = utils.create_corr_data(corr, data_min, data_max)

# Plot the correlated data
utils.plot_data(X, corr)

![png](output_1_0.png)


# 均值归一化

在向数据应用 PCA 之前，需要均值归一化数据。均值归一化将使数据均匀地分布在某个范围内，并以零居中。所以所有数据点的平均值接近 0。以下代码使用 `utils` 模块的 `utils.mean_normalize_data()` 函数归一化我们在上面创建的数据。我们将再次使用 `utils` 模块的  `utils.plot_data()` 函数绘制数据。在绘制均值归一化的数据后，我们将发现数据现在均匀分布在原点 `(0,0)` 周围。这很合理，因为所有数据点的平均值应该为 0。

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

import utils

# Set the default figure size
plt.rcParams['figure.figsize'] = [10.0, 6.0]

# Mean normalize X
X_norm = utils.mean_normalize_data(X)

# Plot the mean normalized correlated data
utils.plot_data(X_norm, corr)

![png](output_3_0.png)


# PCA 简介

我们再回到之前看到的示例。我们有一些接近一条直线的二维数据，我们希望通过将数据点投射到一条直线上，将此数据从二维降到一维。但是如何找到投射数据的最佳直线？这条直线会最小化数据点到投射点的距离平方和。注意，这些投射距离与直线正交，不是像线性回归那样垂直。此外，我们将数据点到投射点的距离称为*投射误差*。知道什么是最佳直线后，如何找到这条线？这时候 PCA 就派上用场了。对于这个示例，PCA 会找到一条当数据投射到该直线后，能够最小化投射误差平方和的直线。我们可以使用 PCA 找到将数据投射到其上面的最佳直线。

通常，对于 $N$-维数据，PCA 将找到使数据投射到上面后能够最小化投射误差的更低维表面。更低维表面由一组向量 $v^{(1)}, v^{(2)}, ...v^{(k)}$ 确定，其中 $k$ 是更低维表面的维度，并且 $k<N$。在上述示例中，我们要将数据从二维降到一维，所以 $k=1$，并且更低维表面是一条直线，仅由一个向量 $v^{(1)}$ 确定。这很合理，因为你只需一个向量就能描述直线。同理，将三维数据降到二维时，$k=2$, ，所以需要用两个向量确定将数据投射到上面的平面（二维表面）。

因此，PCA 算法的作用是找到能够最小化投射误差的更低维表面。正如之前所学习的，这些向量对应于数据矩阵 $X$ 的特征向量的子集。我们将这个特征向量子集称为 $X$ 的*主成分*。我们还将第一个主成分定义为与 $X$ 的最大特征值对应的特征向量，将第二个主成分定义为与 $X$ 的第二大特征值对应的特征向量，以此类推。如果 $v^{(1)}, v^{(2)}, ...v^{(N)}$ 是 $X$ 的特征向量，那么 $X$ 的主成分将由 $v^{(1)}, v^{(2)}, ...v^{(k)}$ 的子集决定，其中 $k<N$。注意，$k$ 决定了向上投射数据的更低维表面的维度。

你可以自己编写 PCA 算法，但幸运的是， Scikit-Learn 等很多软件包已经包含能够为你执行 PCA 算法的内置函数。在下个部分，我们将学习如何使用  Scikit-Learn 实现 PCA 算法。


# 通过  Scikit-Learn 应用 PCA

我们可以使用 Scikit-Learn 的 [`PCA()` 类](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)对数据进行主成分分析。Scikit-Learn 的 `PCA()` 类使用一种叫做**奇异值分解 (SVD)** 的技巧计算某个数据集的特征向量和特征值。假设矩阵 $X$ 的形状为 $(M, N)$，SVD 算法将 $X$ 分解为 3 个矩阵 $U, S$ 和 $V$：

\begin{equation}
X = U S V
\end{equation}

$U$ 和 $V$ 矩阵的形状取决于 SVD 算法的实现方式。在使用 Scikit-Learn 的 `PCA()` 类时，$U$ 和 $V$ 矩阵的维度分别为 $(M, P)$ 和 $(P,N)$，其中 $P = \min(M, N)$。$V$ 的行包含 $X$ 的特征向量，$S$ 矩阵是一个对角 $(P,P)$ 矩阵，其中包含 $X$ 的特征值，并按降序排序，即最大特征值将为元素 $S_{11}$，第二大特征值将为元素 $S_{22}$，以此类推。$V$ 中的特征向量按一定顺序排列，其中 $V$ 的第一行存储的特征向量对应于 $S_{11}$ 中的特征值，$V$ 的第二行存储的特征向量对应于 $S_{22}$ 中的特征值，以此类推。

使用 SVD 计算特征向量和特征值后，使用 PCA 降维的下一步是选择要将数据投射到上面的维度大小。这个维度大小由 $k$ 决定，它规定了我们要使用的主成分数量。我们可以设置参数 `n_components=k`（设为选定的 $k$ 值），告诉 `PCA()` 类要返回的主成分数量，如以下代码所示：

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 2)

print('\nPCA Parameters:', pca, '\n')

In [ ]:
PCA Parameters: PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False) 



可以看出，`pca` 包含 PCA 算法将使用的参数。因为我们在上面创建的随机相关数据是二维的，所以 $X$ 最多有 2 个特征向量。我们设为  $k=2$，所以 `PCA()` 类将返回这两个主成分（特征向量）。之所以设为 $k=2$ 是因为我们要在下个部分可视化这两个主成分。如果我们不想可视化这两个主成分，而是直接降维，可以设为 $k=1$，将数据从二维降成一维，就像一开始所演示的那样。

设置 PCA 算法的参数后，现在需要将数据传递给 `PCA()` 类。我们使用 `.fit()` 方法，如下所示：

In [ ]:
pca.fit(X_norm);

`PCA()` 类通过 `.fit()` 方法拟合数据后，会返回一个数组，其中属性 `.components_` 表示主成分，属性 `.singular_values_` 表示对应的一维数组特征值。`PCA()` 类的其他属性包括 `.explained_variance_ratio_` ，表示每个主成分解释的方差百分比。以下代码访问了上述属性并显示属性内容：

In [ ]:
print('\nArray Containing all Principal Components:\n', pca.components_)
print('\nFirst Principal Component:', pca.components_[0])
print('Second Principal Component:', pca.components_[1])
print('\nEigenvalues:', pca.singular_values_)
print('\nPercentage of Variance Explained by Each Principal Component:', pca.explained_variance_ratio_)

In [ ]:
Array Containing all Principal Components:
 [[-0.70710678 -0.70710678]
 [ 0.70710678 -0.70710678]]

First Principal Component: [-0.70710678 -0.70710678]
Second Principal Component: [ 0.70710678 -0.70710678]

Eigenvalues: [42.39465018 14.23705152]

Percentage of Variance Explained by Each Principal Component: [0.89865318 0.10134682]


可以看出，第一个主成分对应的特征值约为 42，第二个主成分对应的特征值约为 14。从 `.explained_variance_ratio_` 属性还可以看出，第一个主成分解释了约 90% 的数据方差，第二个主成分仅解释了约 10% 的方差。通常，特征值最大的主成分会解释大部分的方差。在降维过程中，通常将数据投射到能解释大部分方差的主成分上。例如在此示例中，我们希望将数据投射到第一个主成分上，因为它解释了 90% 的数据方差。要详细了解 Scikit-Learn 的 `PCA()` 类，请参阅此[文档](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)。 

# 可视化主成分

获得主成分后，下面可视化它们。在以下代码中，我们使用 `utils` 模块的 `utils.plot_data_with_pca_comp()` 函数计算随机相关数据的主成分。该函数会执行我们在上面看到的所有步骤，然后绘出生成的主成分以及数据。为了让你不用上下滚动 notebook 来创建新的随机相关数据，我复制了之前的代码，这样你就可以在下面更改随机数据的参数。

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

import utils

# Set the default figure size
plt.rcParams['figure.figsize'] = [10.0, 6.0]

# Set data range
data_min = 10
data_max = 80

# Set the amount of correlation
corr = 0.8

# Plot the data and the principal components   
utils.plot_data_with_pca_comp(corr, data_min, data_max)

![png](output_12_0.png)


# 选择主成分的数量

我们发现，低维表面 $k$ 的维度是 PCA 算法的一个自由参数。在处理低维数据时，$k$ 的值很好设置，例如，对于二维数据，可以设为 $k=1$,，从而将二维数据降成一维。但是，在处理高维数据时，合适的 $k$ 值不太好确定。例如，假设有一个 1,000-维的数据集，$k$ 的最佳值是多少？应该设为 $k=500$ 将数据从 1,000 维降成 500 维吗，或者进一步降成 100 维，设为 $k=100$？

通常，主成分的数量 $k$ 取决于你要保留原始数据多少的方差。通常，选择的 $k$ 可以使原始数据保留 80% 到 99% 的方差，但是特定的应用可以选择更低的百分比。你可以使用 `.explained_variance_ratio_` 属性查看给定的 $k$ 值可以解释的方差百分比。在实践中，你可以让 `PCA() `类返回所有的特征向量，然后将返回 `.explained_variance_ratio_` 属性的数组中的元素相加，直到达到期望的方差保留百分比。例如，如果我们想保留 98% 的方差，那么 $k$ 应满足以下条件：

\begin{equation}
\sum_{i=1}^k P_{i} \geq 0.98
\end{equation}

其中 $P$ 是返回 `.explained_variance_ratio_` 属性的数组。如果手动选择 $k$ 的值，可以使用上述公式判断这个 $k$ 值能够保留的方差百分比。对于高度相关的数据，可以显著降低数据的维度，即使保留 99% 的方差。


# 投射的数据

我们已经知道所有的主成分是什么，现在将使用 `PCA()` 类并设置 `n_components = 1` 以进行降维。找到低维空间对应的向量（主成分）后，降维算法的下个部分是找到投射到该空间的投射值。我们可以使用 `PCA()` 类的 `.transform()` 方法应用降维并将数据点投射到低维空间上。在这个简单示例中，$k=1$，所以数据只投射到一个主成分上面。

在以下代码中，我们应用 PCA 并设置 `n_components = 1` ，然后使用 `.transform()` 方法将数据点投射到一条直线上。最后绘制投射的数据。

In [ ]:
import numpy as np

from sklearn.decomposition import PCA

pca = PCA(n_components = 1)

pca.fit(X_norm);

transformed_data = pca.transform(X_norm)

yvals = np.zeros(1000)

# Plot the data
plt.scatter(transformed_data, yvals, color = 'white', alpha = 0.5, linewidth = 0)
ax = plt.gca()
ax.set_facecolor('lightslategray')
plt.grid()

plt.show()

![png](output_14_0.png)